In [1]:
#!pip install ultralytics==8.0.196

from ultralytics import YOLO

model = YOLO('/home/theperson/YOLO/runs/detect/train5/weights/best.pt')

results = model.predict(source='/home/theperson/YOLO/PD/PD_100017.png', save=False)

#print(results)


/home/theperson/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "

image 1/1 /home/theperson/YOLO/PD/PD_100017.png: 640x640 1 Pons-MidBrain, 18.6ms
Speed: 3.6ms preprocess, 18.6ms inference, 334.7ms postprocess per image at shape (1, 3, 640, 640)


In [7]:
import cv2
import numpy as np

image = cv2.imread("/home/theperson/YOLO/PD/PD_100017.png")
def snr_cal(img):
    results = model.predict(image, conf=0.5)
    

    full_mask = np.zeros(img.shape[:2], dtype=np.uint8)

    for result in results:
        if hasattr(result, 'boxes'):
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
            
                cv2.rectangle(full_mask, (x1, y1), (x2, y2), 255, thickness=cv2.FILLED)


    mask_3channel = cv2.cvtColor(full_mask, cv2.COLOR_GRAY2BGR)

    masked_img = cv2.bitwise_and(img, mask_3channel)
    
    cv2.imshow('Binary Mask', full_mask)

    cv2.imshow('Masked Image', masked_img)
    #cv2.imwrite("/home/theperson/YOLO/PD/Signal.png", masked_img)


    cv2.waitKey(0) 
    cv2.destroyAllWindows()


    masked_pixels = img[full_mask > 0]  

    if masked_pixels.size > 0:
        mean_pixel_value = np.mean(masked_pixels, axis=0)
    else:
        mean_pixel_value = np.array([0, 0, 0])  
#############################################################
    
    inverted_mask = cv2.bitwise_not(full_mask) 


    inverted_mask_3channel = cv2.cvtColor(inverted_mask, cv2.COLOR_GRAY2BGR)

    noise_img = cv2.bitwise_and(img, inverted_mask_3channel)

    cv2.imshow('Inverted Mask (Noise)', inverted_mask)
    cv2.imwrite("/home/theperson/YOLO/PD/BinaryMask_Inverted.png", inverted_mask)


    cv2.imshow('Noise Image', noise_img)
    cv2.imwrite("/home/theperson/YOLO/PD/Noise.png", noise_img)


    cv2.waitKey(0)  
    cv2.destroyAllWindows()


    noise_pixels = img[inverted_mask > 0]  

    if noise_pixels.size > 0:
        std_noise = np.std(noise_pixels, axis=0)
    else:
        std_noise = np.array([0, 0, 0]) 
    
    return mean_pixel_value/std_noise

snr=snr_cal(image)
print("Mean pixel value inside the boxes (RGB):", snr)



0: 640x640 1 Pons-MidBrain, 18.2ms
Speed: 9.7ms preprocess, 18.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


Mean pixel value inside the boxes (RGB): [     2.1531      2.1531      2.1531]


In [6]:
print("Mean pixel value inside the boxes (RGB):", mean_pixel_value)


Mean pixel value inside the boxes (RGB): [     77.469      77.469      77.469]
